In [1]:
# 목표 예측 범위 안에 넣기
# data * std + mean


In [2]:
%matplotlib widget

In [3]:
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
folder = 'data'
# file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
#               ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
#               ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
              ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
             ]

#file_names = [['자동측정망-01-가평-2016.xlsx','자동측정망-01-가평-2017.xlsx','자동측정망-01-가평-2018.xlsx', '자동측정망-01-가평-2019.xlsx', '자동측정망-01-가평-2020.xlsx'], 
#              ['자동측정망-02-의암호-2016.xlsx','자동측정망-02-의암호-2017.xlsx','자동측정망-02-의암호-2018.xlsx','자동측정망-02-의암호-2019.xlsx','자동측정망-02-의암호-2020.xlsx'],
#              ['자동측정망-03-서상-2016.xlsx','자동측정망-03-서상-2017.xlsx','자동측정망-03-서상-2018.xlsx','자동측정망-03-서상-2019.xlsx','자동측정망-03-서상-2020.xlsx'],
#              ['자동측정망-04-화천-2016.xlsx','자동측정망-04-화천-2017.xlsx','자동측정망-04-화천-2018.xlsx','자동측정망-04-화천-2019.xlsx','자동측정망-04-화천-2020.xlsx'],
              #['총량측정망-10-01-조종천3-2016.xlsx','총량측정망-10-01-조종천3-2017.xlsx','총량측정망-10-01-조종천3-2018.xlsx','총량측정망-10-01-조종천3-2019.xlsx','총량측정망-10-01-조종천3-2020.xlsx'],
              #['총량측정망-12-01-청평-2016.xlsx','총량측정망-12-01-청평-2017.xlsx','총량측정망-12-01-청평-2018.xlsx','총량측정망-12-01-청평-2019.xlsx','총량측정망-12-01-청평-2020.xlsx'],
              #['총량측정망-12-01-청평-2016s.xlsx','총량측정망-12-01-청평-2017.xlsx','총량측정망-12-01-청평-2018.xlsx','총량측정망-12-01-청평-2019.xlsx','총량측정망-12-01-청평-2020.xlsx'],
              #['총량측정망-12-02-가평천3-2016.xlsx','총량측정망-12-02-가평천3-2017.xlsx','총량측정망-12-02-가평천3-2018.xlsx','총량측정망-12-02-가평천3-2019.xlsx','총량측정망-12-02-가평천3-2020.xlsx'],
              #['총량측정망-12-03-춘성교-2016.xlsx','총량측정망-12-03-춘성교-2017.xlsx','총량측정망-12-03-춘성교-2018.xlsx','총량측정망-12-03-춘성교-2019.xlsx','총량측정망-12-03-춘성교-2020.xlsx'],
              #['총량측정망-20-01-소양강2-2016.xlsx','총량측정망-20-01-소양강2-2017.xlsx','총량측정망-20-01-소양강2-2018.xlsx','총량측정망-20-01-소양강2-2019.xlsx','총량측정망-20-01-소양강2-2020.xlsx'],
              #['총량측정망-34-01-춘천A-2016.xlsx','총량측정망-34-01-춘천A-2017.xlsx','총량측정망-34-01-춘천A-2018.xlsx','총량측정망-34-01-춘천A-2019.xlsx','총량측정망-34-01-춘천A-2020.xlsx'],
              #['총량측정망-40-01-화천-2016.xlsx','총량측정망-40-01-화천-2017.xlsx','총량측정망-40-01-화천-2018.xlsx','총량측정망-40-01-화천-2019.xlsx','총량측정망-40-01-화천-2020.xlsx']
         #    ]



day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

stride = len(file_names)

for loc in range(len(file_names)):
    
    df_loc = []
    print(len(file_names[loc]))
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].iloc[:, 2:])
#     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
#     df.append(df_full[loc].iloc[:, 2:11])
    
    date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
    
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)
        

4
가평_2016.xlsx
가평_2017.xlsx
가평_2018.xlsx
가평_2019.xlsx
4
의암호_2016.xlsx
의암호_2017.xlsx
의암호_2018.xlsx
의암호_2019.xlsx


In [5]:
#df[0]

In [6]:
#df[0]

In [7]:
# normalize data

df_all = pd.concat(df)
df_all

train_mean = df_all.mean()
train_std = df_all.std()
for i in range(len(file_names)):
    df[i] = (df[i]-train_mean)/train_std

In [8]:
train_mean, train_std


(수온          1.442927e+01
 수소이온농도      7.338229e+00
 전기전도도       1.067481e+02
 용존산소        1.051423e+01
 탁도          2.159194e+00
 총유기탄소       1.680518e+00
 총질소         1.717054e+00
 총인          7.725232e-03
 클로로필-a      1.064802e+01
 미생물_독성지수   -4.499445e-02
 Day sin     6.037379e-15
 Day cos     2.747517e-15
 Year sin   -5.681266e-08
 Year cos    2.053380e-05
 dtype: float64,
 수온           7.397900
 수소이온농도       0.485797
 전기전도도       13.159411
 용존산소         1.806036
 탁도           5.437522
 총유기탄소        0.368691
 총질소          0.382042
 총인           0.007932
 클로로필-a      10.570897
 미생물_독성지수     1.615261
 Day sin      0.707112
 Day cos      0.707112
 Year sin     0.707105
 Year cos     0.707119
 dtype: float64)

In [9]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,-1.342174,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.782384,-0.21750,-0.468079,0.337403,-1.999582e-12,1.414203,-0.003771,1.414155
1,-1.369209,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.023195,-0.59571,-0.496459,0.275494,3.660228e-01,1.366016,-0.002757,1.414157
2,-1.355692,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.167159,-0.59571,-0.496459,0.337403,7.071017e-01,1.224736,-0.001743,1.414159
3,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,2.002255,-0.72178,-0.496459,2.937603,9.999929e-01,0.999993,-0.000730,1.414160
4,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.926347,-0.59571,-0.496459,-1.519882,1.224736e+00,0.707102,0.000284,1.414160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.463830,-0.490387,0.627070,1.154889,-0.194794,0.324071,0.002475,-0.72178,1.140110,0.089765,-1.366016e+00,0.366023,-0.008109,1.414137
35060,-1.477348,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.023416,-0.84785,1.064430,0.337403,-1.224736e+00,0.707102,-0.007096,1.414142
35061,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.054826,-0.72178,1.007671,0.585041,-9.999929e-01,0.999993,-0.006082,1.414147
35062,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.015563,-0.72178,1.026590,0.461222,-7.071017e-01,1.224736,-0.005068,1.414151


In [10]:
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [11]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [12]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [13]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [14]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [15]:
# not used
# original make_dataset code
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

#WindowGenerator.make_dataset = make_dataset

In [147]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example2', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example2 = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2

In [17]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [18]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [19]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [20]:
def normalization (data, parameters=None):
  '''Normalize data in [0, 1] range.
  
  Args:
    - data: original data
  
  Returns:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  '''

  # Parameters
  _, dim = data.shape
  norm_data = data.copy()

  if parameters is None:

    # MixMax normalization
    min_val = np.zeros(dim)
    max_val = np.zeros(dim)
   
    # For each dimension
    for i in range(dim):
      min_val[i] = np.nanmin(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
      max_val[i] = np.nanmax(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)

    # Return norm_parameters for renormalization
    norm_parameters = {'min_val': min_val,
                       'max_val': max_val}
  else:
    min_val = parameters['min_val']
    max_val = parameters['max_val']

    # For each dimension
    for i in range(dim):
      norm_data[:,i] = norm_data[:,i] - min_val[i]
      norm_data[:,i] = norm_data[:,i] / (max_val[i] + 1e-6)

    norm_parameters = parameters

  return norm_data, norm_parameters

In [21]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        
        #print(data)
        
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
        #print(group)
        n = np.max(group)
        #print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        #print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [22]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)

miss_data file saved


In [23]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [24]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)
            
        
        # normlize for spam
        if normalize:
            self.data, norm_param = normalization(self.data)
        #print(norm_param)
        
        # Define mask matrix
        if miss_pattern is None:
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            self.miss = MissData(load_dir='save')
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

dgen = GainDataGenerator(df)

In [25]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [26]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

In [27]:
train_df = df_all
val_df = df_all
test_df = df_all

In [28]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

In [29]:
wide_window.example[0].shape

TensorShape([128, 120, 14])

In [30]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,-1.342174,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.782384,-0.21750,-0.468079,0.337403,-1.999582e-12,1.414203,-0.003771,1.414155
1,-1.369209,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.023195,-0.59571,-0.496459,0.275494,3.660228e-01,1.366016,-0.002757,1.414157
2,-1.355692,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.167159,-0.59571,-0.496459,0.337403,7.071017e-01,1.224736,-0.001743,1.414159
3,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,2.002255,-0.72178,-0.496459,2.937603,9.999929e-01,0.999993,-0.000730,1.414160
4,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.926347,-0.59571,-0.496459,-1.519882,1.224736e+00,0.707102,0.000284,1.414160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.463830,-0.490387,0.627070,1.154889,-0.194794,0.324071,0.002475,-0.72178,1.140110,0.089765,-1.366016e+00,0.366023,-0.008109,1.414137
35060,-1.477348,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.023416,-0.84785,1.064430,0.337403,-1.224736e+00,0.707102,-0.007096,1.414142
35061,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.054826,-0.72178,1.007671,0.585041,-9.999929e-01,0.999993,-0.006082,1.414147
35062,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.015563,-0.72178,1.026590,0.461222,-7.071017e-01,1.224736,-0.005068,1.414151


In [31]:
wide_window.plot(plot_col='총유기탄소')
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  2173


In [32]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = ~isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:7], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plt.figure(figsize=(9,10))
n = wide_window.dg.data_m.shape[0]
train = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(wide_window.dg.data_m[i*train:(i+1)*train, 0:7], aspect='auto')
    plt.yticks([])
#plt.imshow(wide_window.dg.data[0:100])
#plt.imshow(wide_window.dg.data_m[800:900], aspect='auto')
#print(wide_window.dg.data[0:50])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [34]:
val_performance = {}
performance = {}

In [35]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [36]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [37]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

gain.save(save_dir='save')

Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 100.1420 - disc_loss: 0.7233 - rmse: 0.9510 - val_loss: 0.8946
Epoch 2/2000
1/1 [==============================] - 0s 176ms/step - gen_loss: 86.5569 - disc_loss: 0.4836 - rmse: 0.8754 - val_loss: 0.7773
Epoch 3/2000
1/1 [==============================] - 0s 173ms/step - gen_loss: 71.1541 - disc_loss: 0.5497 - rmse: 0.9537 - val_loss: 0.7095
Epoch 4/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 63.8604 - disc_loss: 0.3957 - rmse: 0.7419 - val_loss: 0.6889
Epoch 5/2000
1/1 [==============================] - 0s 175ms/step - gen_loss: 54.2554 - disc_loss: 0.3974 - rmse: 0.7584 - val_loss: 0.6260
Epoch 6/2000
1/1 [==============================] - 0s 175ms/step - gen_loss: 47.9821 - disc_loss: 0.4058 - rmse: 0.9029 - val_loss: 0.5841
Epoch 7/2000
1/1 [==============================] - 0s 174ms/step - gen_loss: 42.7400 - disc_loss: 0.3898 - rmse: 0.7100 - val_loss: 0.5920
Epoch 8/2000
1/1 [====

1/1 [==============================] - 0s 173ms/step - gen_loss: 8.3926 - disc_loss: 0.3321 - rmse: 0.3203 - val_loss: 0.4688
Epoch 60/2000
1/1 [==============================] - 0s 173ms/step - gen_loss: 8.2087 - disc_loss: 0.3278 - rmse: 0.4320 - val_loss: 0.3312
Epoch 61/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 9.2800 - disc_loss: 0.3227 - rmse: 0.4428 - val_loss: 0.4729
Epoch 62/2000
1/1 [==============================] - 0s 172ms/step - gen_loss: 9.0096 - disc_loss: 0.3130 - rmse: 0.4394 - val_loss: 0.4541
Epoch 63/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 9.3022 - disc_loss: 0.3106 - rmse: 0.6411 - val_loss: 0.3557
Epoch 64/2000
1/1 [==============================] - 0s 172ms/step - gen_loss: 8.3513 - disc_loss: 0.3220 - rmse: 0.7358 - val_loss: 0.4751
Epoch 65/2000
1/1 [==============================] - 0s 173ms/step - gen_loss: 7.9421 - disc_loss: 0.3237 - rmse: 0.5964 - val_loss: 0.6955
Epoch 66/2000
1/1 [===============

1/1 [==============================] - 0s 180ms/step - gen_loss: 7.7007 - disc_loss: 0.2510 - rmse: 0.3280 - val_loss: 0.3270
Epoch 118/2000
1/1 [==============================] - 0s 174ms/step - gen_loss: 7.5562 - disc_loss: 0.2464 - rmse: 0.3387 - val_loss: 0.3777
Epoch 119/2000
1/1 [==============================] - 0s 359ms/step - gen_loss: 6.3440 - disc_loss: 0.2438 - rmse: 0.3174 - val_loss: 0.3013
Epoch 120/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 9.0255 - disc_loss: 0.2480 - rmse: 0.3026 - val_loss: 0.2872
Epoch 121/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 9.1904 - disc_loss: 0.2519 - rmse: 0.3044 - val_loss: 0.2742
Epoch 122/2000
1/1 [==============================] - 0s 172ms/step - gen_loss: 7.4934 - disc_loss: 0.2459 - rmse: 0.3024 - val_loss: 0.2981
Epoch 123/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 8.8130 - disc_loss: 0.2456 - rmse: 0.3023 - val_loss: 0.3038
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 169ms/step - gen_loss: 6.4984 - disc_loss: 0.2233 - rmse: 0.5684 - val_loss: 0.3132
Epoch 176/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 6.0667 - disc_loss: 0.2284 - rmse: 0.2989 - val_loss: 0.2670
Epoch 177/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 5.9043 - disc_loss: 0.2265 - rmse: 0.4045 - val_loss: 0.2931
Epoch 178/2000
1/1 [==============================] - 0s 172ms/step - gen_loss: 6.6037 - disc_loss: 0.2372 - rmse: 0.4288 - val_loss: 0.3061
Epoch 179/2000
1/1 [==============================] - 0s 177ms/step - gen_loss: 6.4014 - disc_loss: 0.2274 - rmse: 0.3869 - val_loss: 0.2907
Epoch 180/2000
1/1 [==============================] - 0s 172ms/step - gen_loss: 6.3340 - disc_loss: 0.2176 - rmse: 0.6377 - val_loss: 0.3121
Epoch 181/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 6.3657 - disc_loss: 0.2160 - rmse: 0.4024 - val_loss: 0.3307
Epoch 182/2000
1/1 [========

1/1 [==============================] - 0s 180ms/step - gen_loss: 5.5254 - disc_loss: 0.1894 - rmse: 0.3087 - val_loss: 0.3081
Epoch 234/2000
1/1 [==============================] - 0s 186ms/step - gen_loss: 7.2316 - disc_loss: 0.2044 - rmse: 0.3388 - val_loss: 0.2909
Epoch 235/2000
1/1 [==============================] - 0s 180ms/step - gen_loss: 6.8804 - disc_loss: 0.1978 - rmse: 0.3184 - val_loss: 0.2924
Epoch 236/2000
1/1 [==============================] - 0s 180ms/step - gen_loss: 5.3855 - disc_loss: 0.1890 - rmse: 0.2949 - val_loss: 0.2842
Epoch 237/2000
1/1 [==============================] - 0s 178ms/step - gen_loss: 6.2212 - disc_loss: 0.1936 - rmse: 0.2946 - val_loss: 0.3107
Epoch 238/2000
1/1 [==============================] - 0s 393ms/step - gen_loss: 5.3959 - disc_loss: 0.1938 - rmse: 0.2859 - val_loss: 0.2805
Epoch 239/2000
1/1 [==============================] - 0s 193ms/step - gen_loss: 7.1023 - disc_loss: 0.1976 - rmse: 0.2911 - val_loss: 0.2724
Epoch 240/2000
1/1 [========

...

1/1 [==============================] - 0s 369ms/step - gen_loss: 1.9563 - disc_loss: 0.1246 - rmse: 0.2645 - val_loss: 0.2551
Epoch 1445/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 1.7812 - disc_loss: 0.1269 - rmse: 0.2283 - val_loss: 0.2745
Epoch 1446/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 2.0375 - disc_loss: 0.1315 - rmse: 0.2450 - val_loss: 0.2615
Epoch 1447/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 2.0215 - disc_loss: 0.1288 - rmse: 0.2664 - val_loss: 0.2496
Epoch 1448/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 2.0378 - disc_loss: 0.1268 - rmse: 0.2431 - val_loss: 0.2640
Epoch 1449/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 2.0070 - disc_loss: 0.1256 - rmse: 0.2541 - val_loss: 0.2681
Epoch 1450/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 1.9324 - disc_loss: 0.1216 - rmse: 0.2418 - val_loss: 0.2620
Epoch 1451/2000
1/1 [=

1/1 [==============================] - 0s 166ms/step - gen_loss: 1.9164 - disc_loss: 0.1332 - rmse: 0.2667 - val_loss: 0.3540
Epoch 1502/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 1.8630 - disc_loss: 0.1372 - rmse: 0.2693 - val_loss: 0.4599
Epoch 1503/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 2.0252 - disc_loss: 0.1353 - rmse: 0.2708 - val_loss: 0.5295
Epoch 1504/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 2.1194 - disc_loss: 0.1329 - rmse: 0.2597 - val_loss: 0.4514
Epoch 1505/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 1.8227 - disc_loss: 0.1366 - rmse: 0.2607 - val_loss: 0.5571
Epoch 1506/2000
1/1 [==============================] - 0s 175ms/step - gen_loss: 1.9327 - disc_loss: 0.1334 - rmse: 0.2802 - val_loss: 0.3525
Epoch 1507/2000
1/1 [==============================] - 0s 177ms/step - gen_loss: 1.9268 - disc_loss: 0.1328 - rmse: 0.2761 - val_loss: 0.2405
Epoch 1508/2000
1/1 [=

1/1 [==============================] - 0s 166ms/step - gen_loss: 1.8864 - disc_loss: 0.1315 - rmse: 0.2647 - val_loss: 0.3318
Epoch 1559/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 1.7976 - disc_loss: 0.1299 - rmse: 0.2495 - val_loss: 0.3741
Epoch 1560/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 1.8435 - disc_loss: 0.1368 - rmse: 0.2430 - val_loss: 0.2679
Epoch 1561/2000
1/1 [==============================] - 0s 163ms/step - gen_loss: 1.7769 - disc_loss: 0.1313 - rmse: 0.2658 - val_loss: 0.3833
Epoch 1562/2000
1/1 [==============================] - 0s 362ms/step - gen_loss: 1.7925 - disc_loss: 0.1342 - rmse: 0.2659 - val_loss: 0.3621
Epoch 1563/2000
1/1 [==============================] - 0s 164ms/step - gen_loss: 2.0365 - disc_loss: 0.1241 - rmse: 0.2744 - val_loss: 0.2763
Epoch 1564/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 1.8929 - disc_loss: 0.1235 - rmse: 0.2629 - val_loss: 0.3013
Epoch 1565/2000
1/1 [=

1/1 [==============================] - 0s 166ms/step - gen_loss: 2.2238 - disc_loss: 0.1289 - rmse: 0.5053 - val_loss: 0.4097
Epoch 1616/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 2.1942 - disc_loss: 0.1308 - rmse: 0.4076 - val_loss: 0.2866
Epoch 1617/2000
1/1 [==============================] - 0s 177ms/step - gen_loss: 1.9210 - disc_loss: 0.1297 - rmse: 0.4114 - val_loss: 0.2761
Epoch 1618/2000
1/1 [==============================] - 0s 172ms/step - gen_loss: 1.9374 - disc_loss: 0.1295 - rmse: 0.4225 - val_loss: 0.2992
Epoch 1619/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 2.0313 - disc_loss: 0.1292 - rmse: 0.5388 - val_loss: 0.3819
Epoch 1620/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 1.8255 - disc_loss: 0.1296 - rmse: 0.3550 - val_loss: 0.3677
Epoch 1621/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 2.0213 - disc_loss: 0.1283 - rmse: 0.5272 - val_loss: 0.5424
Epoch 1622/2000
1/1 [=

1/1 [==============================] - 0s 163ms/step - gen_loss: 1.9495 - disc_loss: 0.1306 - rmse: 0.3460 - val_loss: 0.2379
Epoch 1673/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 2.1532 - disc_loss: 0.1267 - rmse: 0.2964 - val_loss: 0.3388
Epoch 1674/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 1.9655 - disc_loss: 0.1302 - rmse: 0.2503 - val_loss: 0.4523
Epoch 1675/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 1.8440 - disc_loss: 0.1283 - rmse: 0.2705 - val_loss: 0.4209
Epoch 1676/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 1.9083 - disc_loss: 0.1310 - rmse: 0.2610 - val_loss: 0.3874
Epoch 1677/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 1.8670 - disc_loss: 0.1346 - rmse: 0.2719 - val_loss: 0.2872
Epoch 1678/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 1.7855 - disc_loss: 0.1269 - rmse: 0.2403 - val_loss: 0.2636
Epoch 1679/2000
1/1 [=

**학습 loss history 출력**

In [38]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

성능 측정

In [39]:
gain.evaluate(wide_window.test.repeat(), steps=100)

100/100 [==============================] - 3s 32ms/step - loss: 0.2614


0.26137664914131165

샘플 prediction 출력

In [40]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
norm_df = pd.concat(df,axis=0)

In [42]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

70128
(120, 14)
120
x.shape = (70080, 14)
x_reshape.shape = (584, 120, 14)
136398
y_true.shape= (70080, 14)
136398


(1, 120, 14)

In [43]:
# zero loss is normal because there is no ground truth in the real dataset
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

19/19 [==============================] - 0s 6ms/step - loss: 0.0000e+00


0.0

In [44]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [45]:
print(total_n)
print(n)
print(total_n-n)

70128
70080
48


In [46]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(70080, 14) (120, 14)
(70128, 14)


In [47]:
y_pred[~np.isnan(data)] = np.nan

In [48]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

70128
unit_shape= (120, 14)
120
n= 70080
(35040, 10)
(35040, 10)


## 예측 

In [50]:
total_no = oris[0].shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gans[0][train_slice])
val_df = pd.DataFrame(gans[0][val_slice])
test_df = pd.DataFrame(gans[0][test_slice])

train_ori_df = pd.DataFrame(oris[0][train_slice])
val_ori_df = pd.DataFrame(oris[0][val_slice])
test_ori_df = pd.DataFrame(oris[0][test_slice])

num_features = train_df.shape[1]
out_num_features = num_features

In [51]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
print(oris[0].shape, gans[0].shape)
print(oris[1].shape, gans[1].shape)
print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
out_features = [6]
out_num_features = len(out_features)
out_num_features


(35040, 14) (35040, 14)
(35040, 10) (35040, 10)
(35040, 24) (35040, 24)


1

In [52]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [53]:
OUT_STEPS = 24*3
wdg = WaterDataGenerator(train_df,
                         batch_size=128,
                         input_width = 24*7,
                         label_width = OUT_STEPS,
                         shift = OUT_STEPS,
                         out_num_features = out_num_features
                        )

In [54]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-1.342174,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.782384,-0.21750,-0.468079,0.337403,...,1.414204,-1.999581e-12,1.414155,-0.003771,0.070886,-0.078693,-1.531417,0.324341,0.703062,-0.218389
1,-1.369209,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.023195,-0.59571,-0.496459,0.275494,...,1.366016,3.660228e-01,1.414157,-0.002757,0.033358,-0.078693,-1.531417,0.324341,0.703062,-0.218389
2,-1.355692,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.167159,-0.59571,-0.496459,0.337403,...,1.224736,7.071018e-01,1.414159,-0.001743,0.028610,-0.078693,-1.531417,0.268971,0.703062,-0.218389
3,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,2.002255,-0.72178,-0.496459,2.937603,...,0.999993,9.999928e-01,1.414160,-0.000730,0.018624,-0.078693,-1.544935,0.268971,0.703062,-0.218389
4,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.926347,-0.59571,-0.496459,-1.519882,...,0.707102,1.224736e+00,1.414160,0.000284,0.015744,-0.078693,-1.544935,0.324341,0.703062,-0.218389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,0.293425,-0.902082,0.627070,-0.672317,0.007504,0.324071,0.913370,0.16071,-0.401860,-0.095963,...,0.366023,-1.366016e+00,0.412794,-1.352621,-0.136031,-0.696235,0.158251,-0.340098,-0.436807,0.595301
24524,0.279908,-1.107929,0.551079,-0.783057,-0.010886,0.324071,0.994513,0.03464,-0.439699,-0.343601,...,0.707102,-1.224736e+00,0.413763,-1.352324,-0.040539,-0.696235,0.158251,-0.340098,-0.436807,0.324071
24525,0.279908,-1.107929,0.551079,-0.783057,-0.010886,0.324071,1.004983,0.16071,-0.439699,-0.343601,...,0.999993,-9.999928e-01,0.414732,-1.352028,-0.026767,-0.696235,0.158251,-0.340098,-0.360816,0.324071
24526,0.279908,-1.107929,0.551079,-0.783057,-0.010886,0.324071,1.012835,0.16071,-0.439699,-0.095963,...,1.224736,-7.071018e-01,0.415701,-1.351730,0.015870,-0.696235,0.144734,-0.340098,-0.360816,0.324071


In [55]:
it = iter(wdg)

In [56]:
x,y = next(it)
x.shape, y.shape

((128, 168, 24), (128, 72, 1))

In [57]:
def make_dataset_water(self, data):
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [58]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

In [59]:
def hour_to_day_mean(array):
    time = 24
    return tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

In [137]:

cond = {
    "총인":[
        [-1000,0.02],
        [0.02,0.04],
        [0.04,0.10],
        [0.10,0.20],
        [0.20,0.30],
        [0.30,0.50],
        [0.50,1000],
    ],
    "총질소":[
        [-1000,0.2],
        [0.2,0.3],
        [0.3,0.4],
        [0.4,0.6],
        [0.6,1.0],
        [1.0,1.5],
        [1.5,1000],
    ],
    "수소이온농도":[
        [6.5 ,8.5],
        [6.5,8.5],
        [6.5,8.5],
        [6.5,8.5],
        [6.0,8.5],
        [6.0,8.5],
        [-1000,1000],
    ],
    "총유기탄소":[
        [-1000 ,2],
        [2,3],
        [3,4],
        [4,5],
        [5,6],
        [6,8],
        [8,1000],
    ],
    "용존산소":[
        [7.5 ,1000],
        [5,7.5],
        [5,7.5],
        [5,7.5],
        [2,5],
        [2,5],
        [-1000,2],
    ],
    "클로로필-a":[
        [-1000 ,5],
        [5,9],
        [9,14],
        [14,20],
        [20,42],
        [42,70],
        [70,1000],
    ],
    
}


def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plot_out_col_index = self.column_indices[plot_out_col]

  if self.label_columns:
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_out_col_index = plot_out_col_index
    
    
  if model is None:
    return
    
  print("column : " + df_all.columns[plot_col])
    
  count1 = 0
  count2 = 0
  for n in range(len(inputs)):
    predictions = model(inputs)
    pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
    pred_temp = pred_temp * train_std[plot_col] + train_mean[plot_col]
    label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    level_label = 0 
    level_pred = 0
    
    for i in range(0,7,1):
      min, max = cond[df_all.columns[plot_col]][i]
      if min <= label_temp[2] and label_temp[2] < max:
        level_label = i
            #print(i, label_temp[2])
      if min <= pred_temp[2] and pred_temp[2] < max:
        level_pred = i
            
    if level_pred != level_label:
        count1 = count1 + 1
    count2 = count2 + 1 
        
  print(100 - count1*100/count2)

  return 100 - count1*100/count2
    


def compa2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  
  plot_out_col_index = self.column_indices[plot_out_col]

  if self.label_columns:
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_out_col_index = plot_out_col_index
    
    
  if model is None:
    return
    
  print("column : " + df_all.columns[plot_col])
    
  #count1 = 0
  #count2 = 0
  mae = 0
  mse = 0
  rmse = 0

  print(inputs.shape)
  print(len(inputs))
  
  len1 = len(inputs)
  
  pred_arr = []
  label_arr = []

    
  for n in range(len1):
    predictions = model(inputs)
    pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
    #pred_temp = pred_temp * train_std[plot_col] + train_mean[plot_col]
    label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
    #label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]

    pred_arr.append(pred_temp[2])
    label_arr.append(label_temp[2])
    
    #for i in range(3):
    #error = label_temp[i] - pred_temp[i]
    error = label_temp - pred_temp
        
    mae = mae + np.absolute(error)
    mse = mse + error**2

  #print(len(pred_arr))
    
  mae = np.average(mae, axis=0)
  mse = np.average(mse, axis=0)
    
  mae = mae / (len1)
  mse = mse / (len1)
  
  rmse = np.sqrt(mse)
    
  print("mae:")  
  print(mae)
    
  print("rmse")  
  print(rmse)
  
    
  #result = mae/len(inputs)
  #print(result)
  return 1


WindowGenerator.compa2 = compa2
    
WindowGenerator.compa = compa





In [139]:
def compa3(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, example=None):
  inputs, labels = example
  
  plot_out_col_index = self.column_indices[plot_out_col]

  if self.label_columns:
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_out_col_index = plot_out_col_index
    
    
  if model is None:
    return
    
  print("column : " + df_all.columns[plot_col])
    
  #count1 = 0
  #count2 = 0
  mae = 0
  mse = 0
  rmse = 0

  print(inputs.shape)
  print(len(inputs))
  
  len1 = len(inputs)
  
  pred_arr = []
  label_arr = []

    
  for n in range(len1):
    predictions = model(inputs)
    pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
    #pred_temp = pred_temp * train_std[plot_col] + train_mean[plot_col]
    label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
    #label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]

    pred_arr.append(pred_temp[2])
    label_arr.append(label_temp[2])
    
    #for i in range(3):
    #error = label_temp[i] - pred_temp[i]
    error = label_temp - pred_temp
        
    mae = mae + np.absolute(error)
    mse = mse + error**2

  #print(len(pred_arr))
    
  mae = np.average(mae, axis=0)
  mse = np.average(mse, axis=0)
    
  mae = mae / (len1)
  mse = mse / (len1)
  
  rmse = np.sqrt(mse)
    
  print("mae:")  
  print(mae)
    
  print("rmse")  
  print(rmse)
  
    
  #result = mae/len(inputs)
  #print(result)
  return 1


WindowGenerator.compa3 = compa3

In [117]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,-1.342174,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.782384,-0.21750,-0.468079,0.337403,-1.999582e-12,1.414203,-0.003771,1.414155
1,-1.369209,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.023195,-0.59571,-0.496459,0.275494,3.660228e-01,1.366016,-0.002757,1.414157
2,-1.355692,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.167159,-0.59571,-0.496459,0.337403,7.071017e-01,1.224736,-0.001743,1.414159
3,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,2.002255,-0.72178,-0.496459,2.937603,9.999929e-01,0.999993,-0.000730,1.414160
4,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.926347,-0.59571,-0.496459,-1.519882,1.224736e+00,0.707102,0.000284,1.414160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.463830,-0.490387,0.627070,1.154889,-0.194794,0.324071,0.002475,-0.72178,1.140110,0.089765,-1.366016e+00,0.366023,-0.008109,1.414137
35060,-1.477348,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.023416,-0.84785,1.064430,0.337403,-1.224736e+00,0.707102,-0.007096,1.414142
35061,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.054826,-0.72178,1.007671,0.585041,-9.999929e-01,0.999993,-0.006082,1.414147
35062,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.015563,-0.72178,1.026590,0.461222,-7.071017e-01,1.224736,-0.005068,1.414151


In [118]:
target_col = '총유기탄소'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
out_features = [target_col_idx]
out_num_features = len(out_features)
out_num_features

1

column : 총유기탄소
(128, 168, 24)
128
128
mae:
0.6049392819404602
rmse
0.8284449049403937


1

In [149]:
multi_window.compa3(last_baseline, plot_col=out_features[0], example = multi_window.example)

column : 총유기탄소
(128, 168, 24)
128
mae:
0.6049392819404602
rmse
0.8284449049403937


1

In [148]:
multi_window.compa3(last_baseline, plot_col=out_features[0], example = multi_window.example2)

column : 총유기탄소
(128, 168, 24)
128
mae:
0.20953422784805298
rmse
0.3998198371704145


1

In [61]:

def to_class(value, colum):
    level_label = 0
    for i in range(0,7,1):
        min1, max1 = cond[colum][i]
        if min1 <= value and value < max1:
            level_label = i
            # 반전
            #level_label = 7 - i
            return level_label
    return level_label

def plot24_class(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    plt.ylim((0,8))
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    input_temp = input_temp.numpy()
    for day in range(input_temp.size):
        #level_label = 0
        #for i in range(0,7,1):
         #   min1, max1 = cond[df_all.columns[plot_col]][i]
          #  if min1 <= input_temp[0] and input_temp[0] < max1:
          #      level_label = i
          #      break; 
        #input_temp[day] = level_label
        input_temp[day] = to_class(input_temp[day], df_all.columns[plot_col])
                
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    label_temp = label_temp.numpy()
    for day in range(label_temp.size):
        label_temp[day] = to_class(label_temp[day], df_all.columns[plot_col])
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
        
      predict_temp = predict_temp.numpy()
      for day in range(predict_temp.size):
          predict_temp[day] = to_class(predict_temp[day], df_all.columns[plot_col])
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24_class = plot24_class

#multi_window.plot24_class(last_baseline, plot_col=out_features[0])

In [62]:

def plot_class(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  #plt.figure(figsize=(10, 8))
  plt.figure()
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  
  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    plt.ylim((0,8))
    
    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    label_temp = label_temp.numpy()
    for day in range(label_temp.size):
        label_temp[day] = to_class(label_temp[day], df_all.columns[plot_col])
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
        
      predict_temp = predict_temp.numpy()
      for day in range(predict_temp.size):
          predict_temp[day] = to_class(predict_temp[day], df_all.columns[plot_col])

    bar_label = [label_temp[v] for v in range(label_temp.size)]
    bar_pred = [predict_temp[v] for v in range(predict_temp.size)]
        
    x = np.arange(3)
    width = 0.3
    plt.bar(x - 0.17, bar_label, width, label='label')
    plt.bar(x + 0.17, bar_pred, width, label='pred')
    plt.xticks(ticks=[0,1,2], labels=[1,2,3],rotation=0)
    if n == 0:
        plt.legend()

WindowGenerator.plot_class = plot_class

In [63]:
df_all.columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '탁도', '총유기탄소', '총질소', '총인', '클로로필-a',
       '미생물_독성지수', 'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [64]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )
#multi_window

In [65]:
MAX_EPOCHS = 400

def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [66]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

**Baseline model**

In [67]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    #print(inputs[:, -1:, 0:1])
    #return tf.tile(inputs[:, -1:, :out_num_features], [1, OUT_STEPS, 1])
    return tf.tile(inputs[:, -1:, (out_features[0]):(out_features[0]+1)], [1, OUT_STEPS, 1])
    #return tf.tile(inputs[:, -1:, out_features[0]:(out_features[1]+1)], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['BaseLine'] = last_baseline.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['BaseLine'] = last_baseline.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
print('val performance =', multi_val_performance['BaseLine'])
print('test performance = ', multi_performance['BaseLine'])


100/100 [==============================] - 6s 62ms/step - loss: 0.0657 - mean_absolute_error: 0.1514
val performance = [0.06523983180522919, 0.15130750834941864]
test performance =  [0.2140199989080429, 0.2323426753282547]


In [68]:
out_features = [8]
out_features

[8]

In [69]:
_performance={}
#multi_window.plot24(last_baseline, plot_col=out_features[0])
#multi_window.plot24(last_baseline, plot_col='총유기탄소')

_performance['base'] = multi_window.compa(last_baseline, plot_col=out_features[0])

column : 클로로필-a
61.71875


In [70]:
multi_window.plot2(last_baseline, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
multi_window.plot24(last_baseline, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
#multi_window.plot24_class(last_baseline, plot_col=out_features[0])
multi_window.plot_class(last_baseline, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Linear**

In [73]:
import IPython
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

#IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
#multi_window.plot(multi_linear_model, plot_col=0)
print('val performance =', multi_val_performance['Linear'])
print('test performance = ', multi_performance['Linear'])

Epoch 1/400
1/1 [==============================] - 0s 372ms/step - loss: 0.6191 - mean_absolute_error: 0.5726 - val_loss: 0.2880 - val_mean_absolute_error: 0.4583
Epoch 2/400
1/1 [==============================] - 0s 154ms/step - loss: 0.7773 - mean_absolute_error: 0.6171 - val_loss: 0.3028 - val_mean_absolute_error: 0.4777
Epoch 3/400
1/1 [==============================] - 0s 157ms/step - loss: 0.7784 - mean_absolute_error: 0.5692 - val_loss: 0.2841 - val_mean_absolute_error: 0.4507
Epoch 4/400
1/1 [==============================] - 0s 152ms/step - loss: 0.6876 - mean_absolute_error: 0.5953 - val_loss: 0.3150 - val_mean_absolute_error: 0.4865
Epoch 5/400
1/1 [==============================] - 0s 156ms/step - loss: 0.9935 - mean_absolute_error: 0.6691 - val_loss: 0.3099 - val_mean_absolute_error: 0.4785
Epoch 6/400
1/1 [==============================] - 0s 156ms/step - loss: 0.4164 - mean_absolute_error: 0.4822 - val_loss: 0.2736 - val_mean_absolute_error: 0.4648
Epoch 7/400
1/1 [=====

1/1 [==============================] - 0s 151ms/step - loss: 0.4431 - mean_absolute_error: 0.4664 - val_loss: 0.2435 - val_mean_absolute_error: 0.3874
Epoch 52/400
1/1 [==============================] - 0s 157ms/step - loss: 0.4824 - mean_absolute_error: 0.4499 - val_loss: 0.2230 - val_mean_absolute_error: 0.3699
Epoch 53/400
1/1 [==============================] - 0s 154ms/step - loss: 0.4056 - mean_absolute_error: 0.4314 - val_loss: 0.1896 - val_mean_absolute_error: 0.3418
Epoch 54/400
1/1 [==============================] - 0s 153ms/step - loss: 0.4452 - mean_absolute_error: 0.4313 - val_loss: 0.2065 - val_mean_absolute_error: 0.3521
Epoch 55/400
1/1 [==============================] - 0s 158ms/step - loss: 0.5877 - mean_absolute_error: 0.4657 - val_loss: 0.1992 - val_mean_absolute_error: 0.3442
Epoch 56/400
1/1 [==============================] - 0s 155ms/step - loss: 0.3513 - mean_absolute_error: 0.4236 - val_loss: 0.2174 - val_mean_absolute_error: 0.3809
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 163ms/step - loss: 0.4101 - mean_absolute_error: 0.3697 - val_loss: 0.2487 - val_mean_absolute_error: 0.3725
Epoch 102/400
1/1 [==============================] - 0s 158ms/step - loss: 0.2087 - mean_absolute_error: 0.3444 - val_loss: 0.2083 - val_mean_absolute_error: 0.3435
Epoch 103/400
1/1 [==============================] - 0s 159ms/step - loss: 0.3986 - mean_absolute_error: 0.3973 - val_loss: 0.2212 - val_mean_absolute_error: 0.3587
Epoch 104/400
1/1 [==============================] - 0s 158ms/step - loss: 0.3462 - mean_absolute_error: 0.3742 - val_loss: 0.2075 - val_mean_absolute_error: 0.3487
Epoch 105/400
1/1 [==============================] - 0s 155ms/step - loss: 0.3307 - mean_absolute_error: 0.3743 - val_loss: 0.2384 - val_mean_absolute_error: 0.3656
Epoch 106/400
1/1 [==============================] - 0s 160ms/step - loss: 0.3249 - mean_absolute_error: 0.4108 - val_loss: 0.2945 - val_mean_absolute_error: 0.4045
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 158ms/step - loss: 0.2436 - mean_absolute_error: 0.3382 - val_loss: 0.1741 - val_mean_absolute_error: 0.3161
Epoch 152/400
1/1 [==============================] - 0s 158ms/step - loss: 0.3874 - mean_absolute_error: 0.3684 - val_loss: 0.2169 - val_mean_absolute_error: 0.3413
Epoch 153/400
1/1 [==============================] - 0s 162ms/step - loss: 0.3407 - mean_absolute_error: 0.3841 - val_loss: 0.2305 - val_mean_absolute_error: 0.3595
Epoch 154/400
1/1 [==============================] - 0s 170ms/step - loss: 0.2774 - mean_absolute_error: 0.3590 - val_loss: 0.2417 - val_mean_absolute_error: 0.3753
Epoch 155/400
1/1 [==============================] - 0s 158ms/step - loss: 0.3055 - mean_absolute_error: 0.3802 - val_loss: 0.2448 - val_mean_absolute_error: 0.3895
Epoch 156/400
1/1 [==============================] - 0s 159ms/step - loss: 0.2581 - mean_absolute_error: 0.3575 - val_loss: 0.2077 - val_mean_absolute_error: 0.3523
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 154ms/step - loss: 0.1975 - mean_absolute_error: 0.3040 - val_loss: 0.2007 - val_mean_absolute_error: 0.3554
Epoch 202/400
1/1 [==============================] - 0s 155ms/step - loss: 0.2632 - mean_absolute_error: 0.3398 - val_loss: 0.3134 - val_mean_absolute_error: 0.4122
Epoch 203/400
1/1 [==============================] - 0s 158ms/step - loss: 0.3202 - mean_absolute_error: 0.3545 - val_loss: 0.1938 - val_mean_absolute_error: 0.3485
Epoch 204/400
1/1 [==============================] - 0s 156ms/step - loss: 0.2895 - mean_absolute_error: 0.3776 - val_loss: 0.2669 - val_mean_absolute_error: 0.3777
Epoch 205/400
1/1 [==============================] - 0s 154ms/step - loss: 0.4041 - mean_absolute_error: 0.3675 - val_loss: 0.2368 - val_mean_absolute_error: 0.3625
Epoch 206/400
1/1 [==============================] - 0s 156ms/step - loss: 0.2819 - mean_absolute_error: 0.3506 - val_loss: 0.2519 - val_mean_absolute_error: 0.3709
Epoch 207/

KeyboardInterrupt: 

In [ ]:
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)

In [ ]:
plot_history(history)

In [ ]:
_performance['Multi'] = multi_window.compa(multi_linear_model, plot_col=out_features[0])

In [ ]:
multi_window.plot2(multi_linear_model, plot_col=out_features[0])

In [ ]:
multi_window.plot24(multi_linear_model, plot_col=out_features[0])

In [ ]:
multi_window.plot24_class(multi_linear_model, plot_col=out_features[0])

**Elman_RNN**

In [ ]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

In [ ]:
plot_history(history)

In [ ]:
_performance['Elman'] = multi_window.compa(elman_model, plot_col=out_features[0])

In [ ]:
multi_window.plot2(elman_model, plot_col=out_features[0])

In [ ]:
multi_window.plot24(elman_model, plot_col=out_features[0])

In [ ]:
multi_window.plot24_class(elman_model, plot_col=out_features[0])

**GRU**

In [ ]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
MAX_EPOCHS = 100

In [ ]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

In [ ]:
plot_history(history)

In [ ]:
_performance['GRU'] = multi_window.compa(gru_model, plot_col=out_features[0])

In [ ]:
multi_window.plot2(gru_model, plot_col=out_features[0])

In [ ]:
multi_window.plot24(gru_model, plot_col=out_features[0])

In [ ]:
multi_window.plot24_class(gru_model, plot_col=out_features[0])

**LSTM**

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

In [ ]:
plot_history(history)

In [ ]:
_performance['LSTM'] = multi_window.compa(multi_lstm_model, plot_col=out_features[0])

In [ ]:
multi_window.plot2(multi_lstm_model, plot_col=out_features[0])

In [ ]:
multi_window.plot24(multi_lstm_model, plot_col=out_features[0])

In [ ]:
multi_window.plot24_class(multi_lstm_model, plot_col=out_features[0])

**CNN model**

In [ ]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
MAX_EPOCHS = 400
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

In [ ]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

In [ ]:
plot_history(history)

In [ ]:
out_features = [4]
out_features[0]

In [ ]:
multi_window.plot2(multi_conv_model, plot_col=out_features[0])

In [ ]:
multi_window.plot24(multi_conv_model, plot_col=out_features[0])

In [ ]:
multi_window.plot24_class(multi_conv_model, plot_col=out_features[0])

**performance**

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

In [ ]:
print(_performance)

In [ ]:
out_features = [1]

_performance['base'] = multi_window.compa(last_baseline, plot_col=out_features[0])
_performance['Multi'] = multi_window.compa(multi_linear_model, plot_col=out_features[0])
_performance['LSTM'] = multi_window.compa(multi_lstm_model, plot_col=out_features[0])
_performance['Elman'] = multi_window.compa(elman_model, plot_col=out_features[0])
_performance['GRU'] = multi_window.compa(gru_model, plot_col=out_features[0])


print(_performance)

In [ ]:
x = np.arange(len(_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in _performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

**Autoregressive model**

In [ ]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)
    
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)
  #print('x =', x)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape

def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the lstm state
  prediction, state = self.warmup(inputs)

  # Insert the first prediction
  predictions.append(prediction)

  # Run the rest of the prediction steps
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  predictions = tf.keras.layers.Lambda(lambda x: x[:, :, out_features[0]:(out_features[0]+len(out_features))])(predictions)
  return predictions

FeedBack.call = call

print('Output shape (batch, time, features): ', feedback_model(multi_window.example[0]).shape)

In [ ]:
history = compile_and_fit(feedback_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['AR LSTM'] = feedback_model.evaluate(multi_window.val)
multi_performance['AR LSTM'] = feedback_model.evaluate(multi_window.test, verbose=0)

print('val performance =', multi_val_performance['AR LSTM'])
print('test performance = ', multi_performance['AR LSTM'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot24(feedback_model, plot_col=out_features[0])

**performance**

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()